# Tutorial Notebook for nisarhdf.nisarRSLCHDF
---

This notebook demonstrates the functionality of `nisarRSLCHDF`, which is at tool for reading NISAR **SLC** products.  Some of its functionality is specifically developed for the Greenland Mapping Project (GrIMP) workflows, but it may also be of general use for working with these products. If nothing else, the source code provides examples of how to unpack these projects.

Note, this class largely serves to unpack and reformat RSLC data. As such, at present, there is no support for data cubes. The **RSLC** products can be large, so it may not work well on machines with limited memory.

## Installation

The easiest way to ensure the code works is to create a custom environment using the *environment.yml* file included in this *nisarhdf* repository. To do so execute the following,

`conda env create -f environment.yml`

Then setup the kernel for the Jupyter notebooks with the command:

`python -m ipykernel install --user --name=nisarcryo`

Then activate the environment:

`conda activate nisarcryo`

Finally, start Jupyter and open the notebooks.

**Even if you want to work with your existing environment, the *environment.yml* file demonstrates which packages must be installed if they are not already present.**


## Python Imports

In [10]:
%load_ext autoreload
%autoreload 2
import nisarhdf
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray
import getpass
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
if not os.path.exists('./output'):
    os.mkdir('./output')

## Open the HDF

Access files via:

- **`s3`** – ODS links (e.g., `s3://nisar-public/...`)
- **`https`** – NISAR sample product suite via ASF (e.g., redirect to signed S3 URL)
- **`local`** – Specify a local path to the HDF5 file.

Create an instance and open the hdf as shown below. For normal products, you shouldn't have to specify the orbit, but while the orbit fields are present in the example file, they are not populated. When you open the hdf, you can specify the precise orbit XMLs if desired, which are release a few weeks after the MOI orbits used in the processing.

In [12]:
files = {'s3': 's3://nisar-st-data-ondemand/ALOS-2-data/INSAR/cryo_sample_products_092024/frame_280/rslc_products/rslc_ALOS2496725280-230804.h5',
         'https': 'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/' \
             'NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5',
         'local': '/Volumes/insar1/ian/NISAR/strips/frame_280/rslc_products/rslc_ALOS2496725280-230804-008.h5'} # Modify to local file path

In [13]:
RSLCFile  = files['https']
RSLCFile

'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5'

Create an instance and open the hdf as shown below. For normal products, you shouldn't have to specify the orbit, but while the orbit fields are present in the example file, they are not populated. When you open the hdf, you can specify the precise orbit XMLs if desired, which are release a few weeks after the MOI orbits used in the processing. Note the **RSLC** files are large, so use `noLoadData=True` to just load the meta data.

In [14]:
%%time
myRSLC = nisarhdf.nisarRSLCHDF()
myRSLC.openHDF(RSLCFile, referenceOrbitXML=None, referenceOrbit=49672, frame=5280, noLoadData=False, page_buf_size=16*1024**3)

[INFO] Resolving redirect: https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5
[INFO] Downloading from: https://d1mv8zhcvry6x4.cloudfront.net/s3-7fdfbcb0ce308dc58d08f97acb1f0590/sds-n-cumulus-prod-nisar-sample-data.s3.us-west-2.amazonaws.com/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5?A-userid=None&Expires=1750205236&Signature=jtkkppi~rG8IAXq-Aih6G-ryPDEK~Llw366Wsz3~YgVw7evdGd-uS-awlZUuqme-6PGgp4SHNB~2t7~TAZf9mvTOUjtsyqN2H2k~gM6jlhsVwEvnoCnPzJTAvW-uR6b7SJD2TdYPOUpntFUI3WjIr1PCOrcV6g8ndMII6Bq5nkhCLPKJS2qZkyirgFstTECy2WQxxDzhLBTzWDCgqz9f7ArvJUbpEvWJrtHjiTG~g9ld7gMU8pNGe4Be3py1dP7RJJoFHyuUqAKT3sFuqkHJGMy4lXjc3RT5W8l96knbOo-6JeDTOlKhvlRz7C8Ag3Gv2Iv

In [15]:
%%time
myRSLC = nisarhdf.nisarRSLCHDF()
myRSLC.openHDF(RSLCFile, referenceOrbitXML=None, referenceOrbit=49672, frame=5280, noLoadData=True, page_buf_size=16*1024**3)
myRSLC.loadData(['HH'], useNumpy=False)

[INFO] Resolving redirect: https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5
[INFO] Downloading from: https://d1mv8zhcvry6x4.cloudfront.net/s3-7fdfbcb0ce308dc58d08f97acb1f0590/sds-n-cumulus-prod-nisar-sample-data.s3.us-west-2.amazonaws.com/RSLC/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5?A-userid=None&Expires=1750205267&Signature=GKwq7Airpi7KBnBdrnDO6FxIGItQ5tyAMgz~K5Di-d7GHYO5keuiFv3pIzJYLEIWdLIkuq8Ms4NYrGFKRUyLevvJagIbn8krCj3Nr4f33QBbOnhAZEwzESjoCpjUI0oVtMVDxhWsZaUWmslM7m4AGMt-nn8af639quIRrhwXL0k5bzC93149T1ZHsm-aqoCaybomBys9dv6uIYGsTU5Qb0YUPb9saL-kcmZOJfIst23eDXjitTKwjXE5mh0UrOjIIY5CtQ7v-xamdJcztJjgt98CIzPz8IaN74DC32gRxfHrUEsaYDsiaIOWFIu~msuiXpl

## RSLC Parameters

A summary of some of the more important parameters for the **RUNW** products can be obtained by.

In [16]:
myRSLC.printParams()

RSLC.referenceOrbit = 49672 
RSLC.frame = 5280 
RSLC.datetime = 2008-10-12 06:09:12 
RSLC.referenceGranule = None
RSLC.SLCNearRange = 743587.3933110968 
RSLC.SLCFarRange = 772526.7340224093 
RSLC.SLCFirstZeroDopplerTime = 22152.0 
RSLC.SLCLastZeroDopplerTime = 22164.999342105264 
RSLC.LookDirection = right 
RSLC.PassType = ascending 
RSLC.Wavelength = None
RSLC.PRF = 1520.0 
RSLC.epsg = None 
RSLC.SLCRangeSize = 6179 
RSLC.SLCAzimuthSize = 19760 
RSLC.SLCRangePixelSize = 4.68425715625 
RSLC.SLCAzimuthPixelSize = 4.484027431226178 
RSLC.SLCZeroDopplerTimeSpacing = 0.0006578947368421052 
RSLC.SLCNearRange = 743587.3933110968 
RSLC.SLCFirstZeroDopplerTime = 22152.0 
RSLC.SLCIncidenceCenter = None

Data Fields
RSLC.HH


## Save RSLC to Binary File

The RSLC data can be dumped to a floating binary (`np.complex64`) or tiff file. This will take several minutes. Binary output can be produced with the following call:

In [17]:
myRSLC.writeData(f'output/RSLC.{myRSLC.referenceOrbit}_{myRSLC.frame}', bands=['HH'], tiff=False)

To save as tiff, run the same command with `tiff=True, driverName='GTiff'`. Using `driverName=GTiff` avoids including pyramids. which may not be needed. For cloudoptimized geotiff, leave the default value.

In [18]:
#!rm output/RSLC.49672_5280.HH